In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import functools

# Some useful utilities
from mcmc_utils_and_plot import scatter_matrix, build_cov_mat, lognormpdf, plot_bivariate_gauss, eval_func_on_grid

def compose(*functions):
    "Compose a list of functions"
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [ ]:
def mh_acceptance_prob(current_target_logpdf,proposed_target_logpdf, current_sample, proposed_sample, proposal_func):
    """Compute the metropolis-hastings accept-reject probability
    
    Inputs
    ------
    current_target_logpdf : float, logpdf at the current sample in the chain f_X(x^{(k)})
    proposed_target_logpdf : float, logpdf at the proposed sample in the chain
    current_sample : (d, ), current sample
    proposed_sample : (d, ), proposed sample
    proposal_func: f(x, y) callable that gives the log probability of y given x
    
    Returns
    -------
    acceptance probability
    """
    
    prop_reverse = proposal_func(proposed_sample, current_sample)
    prop_forward = proposal_func(current_sample, proposed_sample)
    check = proposed_target_logpdf - current_target_logpdf + prop_reverse - prop_forward
    if check < 0:
        return np.exp(check)
    else:
        return 1    

In [ ]:
def mhmcmc(starting_sample, num_samples, target_logpdf, proposal_logpdf, proposal_sampler):
    """Metropolis-Hastings MCMC
    
    Inputs
    ------
    starting_sample: (d, ) the initial sample
    num_sample: positive integer, the number of total samples
    target_logpdf: function(x) -> logpdf of the target distribution
    proposal_logpdf: function (x, y) -> logpdf of proposing y if current sample is x
    proposal_sampler: function (x) -> y, generate a sample if you are currently at x
    
    Returns
    -------
    Samples: (num_samples, d) array of samples
    accept_ratio: ratio of proposed samples that were accepted
    """

    d = starting_sample.shape[0]
    samples = np.zeros((num_samples, d))
    samples[0, :] = starting_sample
    current_target_logpdf = target_logpdf(samples[0, :])
    
    num_accept = 0
    for ii in range(1, num_samples):
        # propose
        proposed_sample = proposal_sampler(samples[ii-1, :])
        proposed_target_logpdf = target_logpdf(proposed_sample)
        
        # determine acceptance probability
        a = mh_acceptance_prob(current_target_logpdf, proposed_target_logpdf, samples[ii-1,:], proposed_sample, proposal_logpdf)
        
        # Accept or reject the sample
        if a == 1: #guaranteed to accept
            samples[ii, :] = proposed_sample
            current_target_logpdf = proposed_target_logpdf
            num_accept += 1
        else:
            u = np.random.rand()
            if u < a: # accept
                samples[ii, :] = proposed_sample
                current_target_logpdf = proposed_target_logpdf
                num_accept += 1
            else: # reject
                samples[ii, :] = samples[ii-1, :]
                
    return samples, num_accept / float(num_samples-1)

In [ ]:
def banana_logpdf(x):
    xArray = np.array([x[0], x[1]-x[0]**2-1])
    d = 2
    trueCovPost = np.array([[1, 0.9], [0.9, 1]])
    # cov = (2.38/np.sqrt(d))**2*trueCovPost
    cov=trueCovPost
    mu = np.array([0, 0])
    x_mu = xArray-mu
    logpdf = np.log((*1/(np.sqrt((2*np.pi)**2*np.linalg.det(cov))))*np.exp(-0.5*np.dot(x_mu).T,np.dot(np.linalg.inv(cov),x_mu)))
    return logpdf

In [ ]:
def plot_banana():
    plt.figure()
    xgrid = np.linspace(-1, 2, 100)
    ygrid = np.linspace(-1, 2, 100)
    XX, YY = np.meshgrid(xgrid, ygrid)
    plt.contourf(XX, YY, 
                 eval_func_on_grid(compose(np.exp, banana_logpdf), 
                                   xgrid, ygrid).T)

plot_banana()